## Generate Sample Data

In [173]:
import os
import pandas as pd
import numpy as np
from faker import Faker
from custom_providers import NYCAddressProvider, NYCPersonProvider, RentalDescriptionProvider, PropertyAmenitiesProvider, ExtendedZipcodeProvider
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import random
from collections import defaultdict
from dateutil.relativedelta import relativedelta
from datetime import datetime
from datetime import date

Setup the custom providers for more relevant sample data, and where the data files will be stored.

In [174]:
fake = Faker()

# configure the custom providers for our fake data elements
fake.add_provider(NYCAddressProvider)
fake.add_provider(NYCPersonProvider)
fake.add_provider(RentalDescriptionProvider)
fake.add_provider(PropertyAmenitiesProvider)
fake.add_provider(ExtendedZipcodeProvider)

# Create the data subdirectory if it doesn't exist
subdirectory = 'data'
os.makedirs(subdirectory, exist_ok=True)

Generate 10 offices for the organization.

In [175]:
# List of New York areas
nyc_areas = [
    'Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island', 
    'Harlem', 'Upper West Side', 'Chelsea', 'Greenwich Village', 
    'SoHo', 'Tribeca', 'Williamsburg', 'Long Island City', 
    'Astoria', 'Flushing'
]

# Ensure we only use unique names from the list
nyc_areas = nyc_areas[:10]

offices_data = []

for i in range(len(nyc_areas)):
    address = fake.nyc_address()
    office_record = {
        'office_name': f'Dream Homes NYC {nyc_areas[i]} Branch',
        'address': address['address'],
        'city': address['city'],
        'state': address['state'],
        'zipcode': address['zipcode'],
        'neighborhood': address['neighborhood'],
        'phone_number': fake.phone_number(),
        'email': fake.company_email()
    }
    offices_data.append(office_record)

offices_df = pd.DataFrame(offices_data)
offices_csv = os.path.join(subdirectory, 'offices.csv')
offices_df.to_csv(offices_csv, index=False)

Generate 250 denormalized employee records, and designate 80% as agents.

In [176]:
employees_data = []
agents_data = []
used_names = set()
specializations = ['Residential', 'Commercial', 'Luxury', 'Investment', 'Leasing', 'Property Management', 'Foreclosures', 'Short Sales']

# Define the desired employment status distribution
employment_status_distribution = {
    'Active': 0.7,
    'Inactive': 0.1,
    'On Leave': 0.1,
    'Terminated': 0.1
}

employment_status_choices = (
    ['Active'] * int(employment_status_distribution['Active'] * 100) +
    ['Inactive'] * int(employment_status_distribution['Inactive'] * 100) +
    ['On Leave'] * int(employment_status_distribution['On Leave'] * 100) +
    ['Terminated'] * int(employment_status_distribution['Terminated'] * 100)
)

for i in range(250):
    address = fake.nyc_address()
    person = fake.nyc_person(used_names)
    office = random.choice(offices_data)  # Select a random office for the employee
    employment_status = random.choice(employment_status_choices)
    termination_date = fake.date_between(start_date='-5y', end_date=date(2024, 6, 30)) if employment_status == 'Terminated' else None
    specialties = ', '.join(random.sample(specializations, random.randint(1, 5))) if i % 2 == 0 else None
    
    employee_record = {
        'name': person['name'],
        'email': person['email'],
        'phone_number': person['phone_number'],
        'date_of_birth': fake.date_of_birth(minimum_age=18, maximum_age=80),
        'address': address['address'],
        'city': address['city'],
        'state': address['state'],
        'zipcode': address['zipcode'],
        'neighborhood': address['neighborhood'],
        'office_phone_number': office['phone_number'],
        'office_email': office['email'],
        'employment_status': employment_status,
        'sales_total': 0,
        'hire_date': fake.date_between(start_date='-5y', end_date=date(2024, 6, 30)),
        'termination_date': termination_date,
        'license_number': fake.bothify(text='???-####-####') if i % 2 == 0 else None,
        'specialties': specialties,
        'rating': round(random.uniform(1, 5), 2) if i % 2 == 0 else None
    }
    employees_data.append(employee_record.copy())

    if random.random() < 0.8:
        agents_data.append(employee_record.copy())

employees_df = pd.DataFrame(employees_data)
employees_csv = os.path.join(subdirectory, 'employees.csv')
employees_df.to_csv(employees_csv, index=False)

agents_df = pd.DataFrame(agents_data)

Generate 1000 denormalized client records, and up to five interactions with agents.

In [177]:
# Static Property Types and Statuses
client_types = ['Corporate', 'Individual', 'Non-Profit', 'Government', 'Small Business', 'VIP']
interaction_types = ['Email', 'Phone Call', 'In-Person Meeting', 'Follow-Up', 'Inquiry']

clients_data = []
client_interactions_data = []
used_names = set()

for _ in range(1000):
    address = fake.nyc_address()
    person = fake.nyc_person(used_names)
    agent = random.choice(agents_data)  # Select a random agent for any interactions
    client_record = {
        'name': person['name'],
        'email': person['email'],
        'phone_number': person['phone_number'],
        'date_of_birth': fake.date_of_birth(minimum_age=18, maximum_age=80),
        'address': address['address'],
        'city': address['city'],
        'state': address['state'],
        'zipcode': address['zipcode'],
        'neighborhood': address['neighborhood'],
        'preferred_contact_method': random.choice(['Email', 'Phone', 'Mail']),
        'notes': fake.text(),
        'client_type': random.choice(client_types)
    }
    clients_data.append(client_record)

    num_interactions = random.randint(0, 5)
    for _ in range(num_interactions):
        interaction_record = {
            'client_name': client_record['name'],
            'client_email': client_record['email'],
            'client_date_of_birth': client_record['date_of_birth'],
            'agent_name': agent['name'],
            'agent_email': agent['email'],
            'agent_date_of_birth': agent['date_of_birth'],
            'interaction_date': fake.date_between(start_date='-5y', end_date=date(2024, 6, 30)),
            'interaction_type': random.choice(interaction_types),
            'notes': fake.text()
        }
        client_interactions_data.append(interaction_record)

clients_df = pd.DataFrame(clients_data)
clients_csv = os.path.join(subdirectory, 'clients.csv')
clients_df.to_csv(clients_csv, index=False)

client_interactions_df = pd.DataFrame(client_interactions_data)
client_interactions_csv = os.path.join(subdirectory, 'client_interactions.csv')
client_interactions_df.to_csv(client_interactions_csv, index=False)

Generate 20000 denormalized property records, randomize the property_type, property_status, and listing_agent assignments.

In [178]:
# Static Property Types and Statuses
property_types = ['Apartment', 'Townhouse', 'Condo', 'Villa', 'Studio']
property_statuses = ['Pending', 'Reserved', 'Sold', 'Listed', 'Unavailable']

# Define realistic price ranges based on property type
price_ranges = {
    'Apartment': (200000, 500000),
    'Townhouse': (300000, 800000),
    'Condo': (250000, 600000),
    'Villa': (600000, 1500000),
    'Studio': (150000, 400000)
}

# Define realistic rental price ranges based on property type
rental_price_ranges = {
    'Apartment': (1500, 4000),
    'Townhouse': (2500, 7000),
    'Condo': (1800, 5000),
    'Villa': (4000, 12000),
    'Studio': (1000, 3000)
}

properties_data = []
property_reviews_data = []
property_images_data = []

for _ in range(20000):
    address = fake.nyc_address()
    agent = random.choice(agents_data)  # Select a random agent for the listing agent
    
    # Determine listing type (70% rentals, 30% sales)
    if random.random() < 0.7:
        listing_type = 'rent'
        property_type = random.choice(property_types)
        min_price, max_price = rental_price_ranges[property_type]
    else:
        listing_type = 'sale'
        property_type = random.choice(property_types)
        min_price, max_price = price_ranges[property_type]
    
    # Generate a property price within this range
    property_price = round(random.uniform(min_price, max_price), 2)

    # Determine property status with the desired distribution
    status_probability = random.random()
    if status_probability < 0.02:
        status_name = 'Unavailable'
    elif status_probability < 0.04:
        status_name = 'Pending'
    elif status_probability < 0.06:
        status_name = 'Reserved'
    elif status_probability < 0.36:
        status_name = 'Listed'
    else:
        status_name = 'Sold'

    # If the property status is 'Listed', ensure the listing date is within 300 days of the last day of the sample
    if status_name == 'Listed':
        listing_date = fake.date_between(start_date='-300d', end_date=date(2024, 6, 30))
    else:
        listing_date = fake.date_between(start_date='-5y', end_date=date(2024, 6, 30))

    property_record = {
        'address': address['address'],
        'city': address['city'],
        'state': address['state'],
        'zipcode': address['zipcode'],
        'neighborhood': address['neighborhood'],
        'type_name': property_type,
        'status_name': status_name,
        'price': property_price,
        'square_feet': random.randint(500, 5000),
        'number_of_bedrooms': random.randint(1, 5),
        'number_of_bathrooms': random.randint(1, 3),
        'year_built': random.randint(1900, 2021),
        'description': fake.rental_description(),
        'amenities': fake.property_amenities(),
        'listing_date': listing_date,
        'agent_name': agent['name'],
        'agent_email': agent['email'],
        'agent_phone': agent['phone_number'],
        'agent_date_of_birth': agent['date_of_birth'],
        'listing_type': listing_type  # Add listing type (rent or sale)
    }
    properties_data.append(property_record)

    # Generate up to 5 property reviews
    num_reviews = random.randint(0, 5)
    for _ in range(num_reviews):
        client = random.choice(clients_data)
        review_record = {
            'property_address': address['address'],
            'property_city': address['city'],
            'property_state': address['state'],
            'property_zipcode': address['zipcode'],
            'client_name': client['name'],
            'client_email': client['email'],
            'client_date_of_birth': client['date_of_birth'],
            'review_date': fake.date_between(start_date='-5y', end_date=date(2024, 6, 30)),
            'rating': random.randint(1, 5),
            'review_text': fake.text()
        }
        property_reviews_data.append(review_record)

    # Generate up to 10 property images
    num_images = random.randint(0, 10)
    for _ in range(num_images):
        image_record = {
            'property_address': address['address'],
            'property_city': address['city'],
            'property_state': address['state'],
            'property_zipcode': address['zipcode'],
            'image_url': fake.image_url(),
            'description': fake.text(),
            'uploaded_date': fake.date_between(start_date='-5y', end_date=date(2024, 6, 30))
        }
        property_images_data.append(image_record)

# Save the dataframes to CSV files
properties_df = pd.DataFrame(properties_data)
properties_csv = os.path.join(subdirectory, 'properties.csv')
properties_df.to_csv(properties_csv, index=False)

property_reviews_df = pd.DataFrame(property_reviews_data)
property_reviews_csv = os.path.join(subdirectory, 'property_reviews.csv')
property_reviews_df.to_csv(property_reviews_csv, index=False)

property_images_df = pd.DataFrame(property_images_data)
property_images_csv = os.path.join(subdirectory, 'property_images.csv')
property_images_df.to_csv(property_images_csv, index=False)

Generate 5000 denormalized event records.

In [180]:
event_types = [
    'Personal Showing', 'Virtual Tour', 'Open House'
]

# Generate Events
events_data = []
for _ in range(5000):
    agent = random.choice(agents_data)          # Select a random agent for the event
    property = random.choice(properties_data)   # Select a random property for the event
    client = random.choice(clients_data)        # Select a random client for the event
    event_record = {
        'type_name': random.choice(event_types),
        'date': fake.date_between(start_date='-5y', end_date=date(2024, 6, 30)),
        'client_attended': random.choice([True, False]),
        'duration': random.randint(1, 2) * 60,
        'client_name': client['name'],
        'client_email': client['email'],
        'client_date_of_birth': client['date_of_birth'],
        'property_address': property['address'],
        'property_city': property['city'],
        'property_state': property['state'],
        'property_zipcode': property['zipcode'],
        'property_neighborhood': property['neighborhood'],
        'agent_name': agent['name'],
        'agent_email': agent['email'],
        'agent_phone': agent['phone_number'],
        'agent_date_of_birth': agent['date_of_birth']
    }
    events_data.append(event_record)

events_df = pd.DataFrame(events_data)
events_csv = os.path.join(subdirectory, 'events.csv')
events_df.to_csv(events_csv, index=False)

Generate 2000 denormalized transaction records.

In [181]:
transaction_types = ['Purchase', 'Sale', 'Rent']

transactions_data = []
for _ in range(2000):
    agent = random.choice(agents_data)
    client = random.choice(clients_data)
    property = random.choice(properties_data)
    transaction_type = random.choice(transaction_types)

    # Ensure the transaction price is within 3% of the property's price
    property_price = property['price']
    min_transaction_price = property_price * 0.97  # 3% below the property price
    max_transaction_price = property_price * 1.03  # 3% above the property price
    
    # Generate a transaction price within this range
    transaction_price = round(random.uniform(min_transaction_price, max_transaction_price), 2)
    
    # Calculate commission based on the listing type
    if property['listing_type'] == 'Rent':
        commission = transaction_price  # Commission equals the transaction price for rentals
    else:
        # Calculate commission as a random percentage of the price (3-5%) for sales
        commission_percentage = random.uniform(3, 5)
        commission = round((commission_percentage / 100) * transaction_price, 2)
    
    # Generate transaction dates
    transaction_date = fake.date_between(start_date='-5y', end_date=date(2024, 6, 30))
    contract_signed_date = fake.date_between(start_date=transaction_date, end_date=transaction_date + timedelta(days=60))
    closing_date = fake.date_between(start_date=transaction_date, end_date=transaction_date + timedelta(days=60))

    transaction_record = {
        'property_address': property['address'],
        'property_city': property['city'],
        'property_state': property['state'],
        'property_zipcode': property['zipcode'],
        'property_neighborhood': property['neighborhood'],
        'client_name': client['name'],
        'client_email': client['email'],
        'client_date_of_birth': client['date_of_birth'],
        'agent_name': agent['name'],
        'agent_email': agent['email'],
        'agent_phone': agent['phone_number'],
        'agent_date_of_birth': agent['date_of_birth'],
        'transaction_type': transaction_type,
        'price': transaction_price,
        'commission': commission,
        'date': transaction_date,
        'contract_signed_date': contract_signed_date,
        'closing_date': closing_date
    }
    transactions_data.append(transaction_record)

transactions_df = pd.DataFrame(transactions_data)
transactions_csv = os.path.join(subdirectory, 'transactions.csv')
transactions_df.to_csv(transactions_csv, index=False)

Generate 1000 business expense records.

In [187]:
expense_types = ['Utilities', 'Marketing', 'Travel', 'Office Supplies', 'Maintenance', 'Technology', 'Training', 'Event', 'Miscellaneous']
non_rent_expense_types = ['Utilities', 'Marketing', 'Travel', 'Office Supplies', 'Maintenance', 'Technology', 'Training', 'Event', 'Miscellaneous']

# Custom distribution for non-rent expenses
non_rent_distribution = {
    'Utilities': 0.1,
    'Marketing': 0.2,
    'Travel': 0.1,
    'Office Supplies': 0.1,
    'Maintenance': 0.1,
    'Technology': 0.1,
    'Training': 0.1,
    'Event': 0.1,
    'Miscellaneous': 0.1
}

# Create a weighted list for non-rent expenses
non_rent_expense_choices = []
for expense_type, weight in non_rent_distribution.items():
    non_rent_expense_choices.extend([expense_type] * int(weight * 100))

business_expenses_data = []
monthly_expenses = defaultdict(float)

# Assign consistent rent amounts to each office
office_rent_amounts = {
    office['office_name']: round(random.uniform(3000, 5000), 2)
    for office in offices_data
}

# Generate rent expenses for each office with a randomly selected start date
current_date = date(2024, 6, 30)
for office in offices_data:
    rent_amount = office_rent_amounts[office['office_name']]
    start_date = fake.date_between(start_date='-5y', end_date=date(2024, 6, 30))
    employee = random.choice(employees_data)
    
    rent_date = start_date
    while rent_date <= current_date:
        if monthly_expenses[rent_date.strftime('%Y-%m')] + rent_amount > 10000:
            rent_date += relativedelta(months=1)
            continue
        
        expense_record = {
            'office_name': office['office_name'],
            'office_address': office['address'],
            'office_city': office['city'],
            'office_state': office['state'],
            'office_zipcode': office['zipcode'],
            'expense_date': rent_date,
            'expense_type': 'Rent',
            'amount': rent_amount,
            'description': '',
            'approved_by_name': employee['name'],
            'approved_by_email': employee['email'],
            'approved_by_phone_number': employee['phone_number']
        }
        business_expenses_data.append(expense_record)
        monthly_expenses[rent_date.strftime('%Y-%m')] += rent_amount
        
        rent_date += relativedelta(months=1)

# Generate 1000 records for other expenses
for _ in range(1000):
    office = random.choice(offices_data)
    employee = random.choice(employees_data)
    expense_type = random.choice(non_rent_expense_choices)
    expense_amount = round(random.uniform(100, 1000), 2)
    expense_date = fake.date_between(start_date='-5y', end_date=date(2024, 6, 30))
    
    if monthly_expenses[expense_date.strftime('%Y-%m')] + expense_amount > 10000:
        continue
    
    expense_record = {
        'office_name': office['office_name'],
        'office_address': office['address'],
        'office_city': office['city'],
        'office_state': office['state'],
        'office_zipcode': office['zipcode'],
        'expense_date': expense_date,
        'expense_type': expense_type,
        'amount': expense_amount,
        'description': '',
        'approved_by_name': employee['name'],
        'approved_by_email': employee['email'],
        'approved_by_phone_number': employee['phone_number']
    }
    business_expenses_data.append(expense_record)
    monthly_expenses[expense_date.strftime('%Y-%m')] += expense_amount

# Save the data to a CSV file
business_expenses_df = pd.DataFrame(business_expenses_data)
business_expenses_csv = os.path.join(subdirectory, 'business_expenses.csv')
business_expenses_df.to_csv(business_expenses_csv, index=False)

## Import Sample Data

Load the generated CSV files to our database using psycopg2. The code will force all connections to the nyc_dream_homes database to be disconnected, remove the database from the instance, and then recreate the database and schema before proceeding with the import.

Note: If psycopg2 is not installed, execute `pip install psycopg2-binary` in your terminal.

### Configuration settings

Change these values if needed for your environment.

In [183]:
# Database connection parameters
dbname = 'nyc_dream_homes'
user = 'postgres'
password = '123'
host = 'localhost'
port = '5432'

### Functions

In [184]:
# Function to execute a single SQL command
def execute_sql_command(conn, command):
    with conn.cursor() as cur:
        cur.execute(command)
        conn.commit()

# Function to drop all active connections to the database
def drop_active_connections(conn, dbname):
    drop_connections_query = f"""
    SELECT pg_terminate_backend(pg_stat_activity.pid)
    FROM pg_stat_activity
    WHERE pg_stat_activity.datname = '{dbname}'
      AND pid <> pg_backend_pid();
    """
    execute_sql_command(conn, drop_connections_query)

# Function to drop and recreate the database
def recreate_database():
    # Connect to the default database to drop and recreate the target database
    conn = psycopg2.connect(
        dbname='postgres',
        user=user,
        password=password,
        host=host,
        port=port
    )
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    drop_active_connections(conn, dbname)
    execute_sql_command(conn, f'DROP DATABASE IF EXISTS {dbname};')
    execute_sql_command(conn, f'CREATE DATABASE {dbname};')
    conn.close()

# Function to run schema creation SQL commands from a file
def run_schema_creation(conn, schema_file_path):
    with open(schema_file_path, 'r') as schema_file:
        schema_sql = schema_file.read()
    with conn.cursor() as cur:
        cur.execute(schema_sql)
        conn.commit()

# Connect to the recreated database
def connect_to_database():
    return psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )

# Function to map Pandas dtypes to PostgreSQL dtypes
def map_dtype(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(dtype):
        return 'NUMERIC'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATE'
    else:
        return 'TEXT'

# Function to drop tables if they already exist
def drop_table_if_exists(table_name, conn):
    drop_table_query = f"DROP TABLE IF EXISTS {table_name};"
    execute_sql_command(conn, drop_table_query)

# Function to create tables based on CSV structure
def create_table_from_csv(csv_path, table_name, conn, date_columns=[]):
    drop_table_if_exists(table_name, conn)
    df = pd.read_csv(csv_path, parse_dates=date_columns)
    col_str = ", ".join([f"{col} {map_dtype(df[col].dtype)}" for col in df.columns])
    create_table_query = f"CREATE TABLE {table_name} ({col_str});"
    execute_sql_command(conn, create_table_query)

# Function to load data from CSV into PostgreSQL
def load_data_to_postgres(csv_path, table_name, conn):
    with conn.cursor() as cur:
        with open(csv_path, 'r') as f:
            cur.copy_expert(f"COPY {table_name} FROM STDIN WITH CSV HEADER", f)
        conn.commit()
    return verify_table(conn, table_name)

# Functions to verify if tables were loaded properly
def verify_table(conn, table_name):
    with conn.cursor() as cur:
        check_table_query = f"SELECT COUNT(*) FROM {table_name};"
        cur.execute(check_table_query)
        count = cur.fetchone()[0]
    return count

def display_results(table_counts):
    for table, count in table_counts.items():
        print(f"Table {table} has {count} records.")

### Rebuild NYC Dream Homes

Recreate the database from scratch removing all previous data, then import the temporary tables from CSV files.

In [188]:

recreate_database()

# Connect to the newly created database
conn = connect_to_database()

# Run schema creation from external SQL file
schema_file_path = '../schema/schema_master.sql'
run_schema_creation(conn, schema_file_path)

# Create temporary tables
create_table_from_csv(clients_csv, 'tmp_clients', conn, date_columns=['date_of_birth'])
create_table_from_csv(employees_csv, 'tmp_employees', conn, date_columns=['date_of_birth', 'hire_date', 'termination_date'])
create_table_from_csv(properties_csv, 'tmp_properties', conn, date_columns=['listing_date', 'agent_date_of_birth'])
create_table_from_csv(events_csv, 'tmp_events', conn, date_columns=['date', 'client_date_of_birth', 'agent_date_of_birth'])
create_table_from_csv(offices_csv, 'tmp_offices', conn, date_columns=[])
create_table_from_csv(transactions_csv, 'tmp_transactions', conn, date_columns=['date', 'client_date_of_birth', 'agent_date_of_birth', 'contract_signed_date', 'closing_date'])
create_table_from_csv(property_reviews_csv, 'tmp_property_reviews', conn, date_columns=['client_date_of_birth', 'review_date'])
create_table_from_csv(property_images_csv, 'tmp_property_images', conn, date_columns=['uploaded_date'])
create_table_from_csv(client_interactions_csv, 'tmp_client_interactions', conn, date_columns=['client_date_of_birth', 'agent_date_of_birth', 'interaction_date'])
create_table_from_csv(business_expenses_csv, 'tmp_business_expenses', conn, date_columns=['expense_date'])

table_counts = {
    'tmp_clients': load_data_to_postgres(clients_csv, 'tmp_clients', conn),
    'tmp_employees': load_data_to_postgres(employees_csv, 'tmp_employees', conn),
    'tmp_properties': load_data_to_postgres(properties_csv, 'tmp_properties', conn),
    'tmp_events': load_data_to_postgres(events_csv, 'tmp_events', conn),
    'tmp_offices': load_data_to_postgres(offices_csv, 'tmp_offices', conn),
    'tmp_transactions': load_data_to_postgres(transactions_csv, 'tmp_transactions', conn),
    'tmp_property_reviews': load_data_to_postgres(property_reviews_csv, 'tmp_property_reviews', conn),
    'tmp_property_images': load_data_to_postgres(property_images_csv, 'tmp_property_images', conn),
    'tmp_client_interactions': load_data_to_postgres(client_interactions_csv, 'tmp_client_interactions', conn),
    'tmp_business_expenses': load_data_to_postgres(business_expenses_csv, 'tmp_business_expenses', conn)
}

# Display the results
display_results(table_counts)

Table tmp_clients has 1000 records.
Table tmp_employees has 250 records.
Table tmp_properties has 20000 records.
Table tmp_events has 5000 records.
Table tmp_offices has 10 records.
Table tmp_transactions has 2000 records.
Table tmp_property_reviews has 49859 records.
Table tmp_property_images has 99328 records.
Table tmp_client_interactions has 2532 records.
Table tmp_business_expenses has 416 records.


## Load Schema via SQL

The following code has been added to streamline the execution of the SQL used to transfer all of the temporary data to the database schema.

The SQL is stored in `load_data.sql` and can be executed directly in PostgreSQL if preferred.

In [189]:

# Function to run schema creation SQL commands from a file
def run_schema_load(conn, load_file_path):
    with open(load_file_path, 'r') as load_file:
        load_sql = load_file.read()
    with conn.cursor() as cur:
        cur.execute(load_sql)
        conn.commit()

# Run final data load from external SQL file
load_file_path = 'load_data.sql'
run_schema_load(conn, load_file_path)

Clean up the temporary tables from the production database.

In [171]:
for table_name in table_counts.keys():
    drop_table_if_exists(table_name, conn)

# Close the connection
conn.close()